Setup

In [3]:

!pip install -q accelerate transformers peft deepspeed trl bitsandbytes datasets flash-attn --upgrade
!pip install -q unsloth
!pip install -q wandb

In [ ]:
with open('deepspeed_zero3_config.yaml', 'w') as f:
    f.write(deepspeed_config)

In [ ]:
with open('train.py', 'w') as f:
    f.write(train_script)  

with open('utils.py', 'w') as f:
    f.write(utils_script)  


In [ ]:
!accelerate launch --config_file "deepspeed_zero3_config.yaml" train.py \
  --seed 42 \
  --model_name_or_path "linyilama/linyilama-1.18-Chat-v1.0" \
  --dataset_name "owl-health/medical-dialogue-to-soap-summary" \
  --chat_template_format "chatml" \
  --add_special_tokens False \
  --append_concat_token False \
  --splits "train,test" \
  --max_seq_length 2048 \
  --num_train_epochs 1 \
  --logging_steps 5 \
  --log_level "info" \
  --logging_strategy "steps" \
  --evaluation_strategy "epoch" \
  --save_strategy "epoch" \
  --bf16 True \
  --packing True \
  --learning_rate 1e-4 \
  --lr_scheduler_type "cosine" \
  --weight_decay 1e-4 \
  --warmup_ratio 0.0 \
  --max_grad_norm 1.0 \
  --output_dir "llama-medical-finetuned" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --gradient_accumulation_steps 2 \
  --gradient_checkpointing True \
  --use_reentrant True \
  --dataset_text_field "text" \
  --use_flash_attn True \
  --use_peft_lora True \
  --lora_r 64 \
  --lora_alpha 16 \
  --lora_dropout 0.1 \
  --lora_target_modules "q_proj,k_proj,v_proj,o_proj,down_proj,up_proj,gate_proj" \
  --use_4bit_quantization True \
  --use_nested_quant True

Training progress

In [ ]:
import time
while True:
    try:
        with open('llama-medical-finetuned/trainer_state.json', 'r') as f:
            import json
            state = json.load(f)
            print(f"Epoch: {state['epoch']:.2f}, Step: {state['step']}/{state['max_steps']}")
        time.sleep(60)
    except:
        print("Training in progress...")
        time.sleep(60)